# Life and death on the lattice

This reports introduces a lattice model for passive and so-called "active" diffusion.

The model consists of particles characterized by a integer-valued position and evolving
at discrete time steps.
The passive particles move by random jumps of step 1 either to the left or to the right.
The active particles possess a velocity of norm 1 and value -1 or +1. At every time step,
they change of direction with probability $p_f$.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import tidynamics
π = np.pi

In [ ]:
def random_sign(n):
    return (-1+2*np.random.randint(0, 2, size=n)).tolist()

def random_flip(n, proba):
    result = np.ones(n, dtype=int)
    result[np.random.random(size=n)<proba] = -1
    return result

def compute_rho(indices, Nx):
    result = np.zeros(Nx, dtype=int)
    for idx in indices:
        result[idx%Nx] += 1
    return result

def compute_rho(indices, Nx):
    return np.histogram(indices, bins=-0.5+np.arange(Nx+1))[0]

def random_move(n, proba_move):
    result = -1+2*np.random.randint(0, 2, size=n)
    result[np.random.random(size=n) > proba_move] = 0
    return result


## The passive model - diffusion

### Model

- $x(t+1) = x(t) + 1$ with probability $p_\mathrm{move}/2$
- $x(t+1) = x(t) - 1$ with probability $p_\mathrm{move}/2$
- $x(t+1) = x(t)$ with probability $1-p_\mathrm{move}$

For $p_\mathrm{move}=1$, the probability density after $t$ steps is
$$P(x(t) = X) \propto e^{-X^2/(2 t)} = e^{-X^2/(4 D t)}$$
which means that the diffusion coefficient is 1/2.

For other values of $p_\mathrm{move} \in [0, 1]$, the diffusion constant
is modified and by rescaling the time we have $D = \frac{p_\mathrm{move}}{2}$

In [ ]:
# Parameters of the lattice

N = 2048    # Number of particles
Nx = 1024    # Number of lattice sites

# Initial condition: delta peak

p_idx = np.ones(N)*(Nx//2)
# half of the particles are shifted by one to avoid sawtooth histograms
p_idx[::2] += 1

plt.figure()

new_rho = []
N_total = []
proba_move = 0.3
D = proba_move / 2
for i in range(512+1):
    p_idx += random_move(len(p_idx), proba_move=proba_move)
    p_idx = np.mod(p_idx, Nx)
    ρ = compute_rho(p_idx, Nx)
    if i>0 and i%128==0:
        l, = plt.plot(ρ)
        plt.plot(N / np.sqrt(D*i*4*π) * np.exp(-(np.arange(Nx)-Nx//2)**2/(4*D*i)), ls='--', color=l.get_color())
        
p_min, p_max = p_idx.min(), p_idx.max()

half_width_max = max(abs(p_min-Nx/2), abs(p_max-Nx/2))

plt.xlim(Nx/2 - half_width_max, Nx/2 + half_width_max)


## Adding life to passive diffusion

We aim here at reproducing locally a chemical kinetics of

$$\frac{d}{dt} {[A]} = k_b [A] - k_d [A]^2$$

where $k_b$ is the birth rate and $k_d$ the death rate.

Death is here modeled as a binary collision process, as this is the appropriate kinetic
model to obtain a $[A]^2$ term in the rate equation.

For the particles, we use the following procedure:

- If $\xi_b < k_b A$ add one particle in the lattice cell.
- If $\xi_d < k_d A^2 / A_0$ remove two particles in the lattice cell.

$\xi_b$ and $\xi_d$ are random numbers in $[0, 1]$.

The quadratic terms means that the cell-wise density must be evaluated at every time
step.

### Chemical kinetics

To model the birth and death process, we use the following reactions:

- $A -> 2A$ with rate $k_b$
- $2A -> 0$ with rate $k_d$

As we work on a lattice, we can use the same method as for multiparticle collision
type fluids proposed by Rohlf, Fraser and Kapral (Comp. Phys. Commun. 2008).

In any given cell, and for low enough rates, the probabilities are as follows:

- $p_b = N_A k_b \tau$ is the probability of a birth
- $p_d = N_A (N_A - 1) k_d \tau /2$ is the probability of two deaths
- The probabilities do not sum to 1, as the "remaininder" is simply the
  probability of no reaction (that should of course go to 1 for small
  values of $\tau$).
  
The chemical kinetics is encoded in the function `react` that will return:
- 0 for no reaction
- 1 for creating a particle
- 2 for removing two particles

After processing all cells, the program below will look for particles belonging to cells
where deletions occur, store their indices and remove them with the `.pop` method (larger
indices first to avoid shifting indices).


In [ ]:
def react(count, k1, k2):
    if count==0:
        return 0
    proba_1 = count*k1
    if count>1:
        proba_2 = count*(count-1)*k2 # Rohlf Eq. (4)
    else:
        proba_2 = 0
    xi = np.random.random()
    reaction = 0
    if xi < proba_1+proba_2:
        xi = np.random.random()*(proba_1+proba_2)
        if xi < proba_1:
            reaction = 1
        else:
            reaction = 2
    return reaction


In [ ]:
# Parameters of the lattice

N = 512    # Number of particles
Nx = 2048    # Number of lattice sites

p_idx = np.random.randint(1024-32, 1024+32, size=N)

plt.figure()

proba_move = 0.0025
k1 = proba_move
ρ_max = 10
k2 = k1/2/ρ_max

new_rho = []
N = len(p_idx)
N_r = [N]
N_total = [N]
BASE = 1024

for i in range(1+6*BASE):
    p_idx += random_move(len(p_idx), proba_move=proba_move)
    p_idx = np.mod(p_idx, Nx)
    ρ = compute_rho(p_idx, Nx)

    new_p = []
    kill_p = []

    for j in range(Nx):
        count = ρ[j]

        reaction = react(count, k1, k2)
        
        if reaction==1:
            # create a particle
            new_p.append(j)
            N += 1
        elif reaction==2:
            # destroy two particles
            kill_p.append(j)
            kill_p.append(j)
            N -= 2
    p_idx = p_idx.tolist()
    kill_indices = []
    for k in kill_p:
        for l, idx in enumerate(p_idx):
            if idx==k and l not in kill_indices:
                kill_indices.append(l)
                break
    kill_indices.sort()
    kill_indices.reverse()
    for k in kill_indices:
        p_idx.pop(k)
    p_idx = np.array([*p_idx, *new_p])
    N_total.append(len(p_idx))
    N_r.append(N)
    ρ = compute_rho(p_idx, Nx)

    if i>0 and i%BASE==0:
        l, = plt.plot(ρ)

p_min, p_max = p_idx.min(), p_idx.max()
half_width_max = max(abs(p_min-Nx/2), abs(p_max-Nx/2))
#plt.xlim(Nx/2 - half_width_max, Nx/2 + half_width_max)

plt.figure()
plt.plot(N_total)
plt.plot(N_r)


### Continuous model

$$\partial_t \rho = D \partial_{xx} \rho + k_b \rho - k_d \rho^2$$

$$\partial_t \rho = D \partial_{xx} \rho + k_b \rho (1 - \frac{k_d}{k_b} \rho)$$

Transform
- $t \to t k_b$
- $D \to D / k_B$

In [ ]:
600/4000

In [ ]:
2*ρ_max*5/np.sqrt(6)*proba_move/2

In [ ]:
D

In [ ]:
proba_move

In [ ]:
t = np.arange(len(N_r))

fit = np.polyfit(t, N_r, 1)

plt.figure()
plt.plot(N_r)
plt.plot(t, np.poly1d(fit)(t))

In [ ]:
fit